# Karaoke Recommender

1. Get a list of songs or artist (Scrape/ cut and paste)
2. Use list to get spotify ID
3. Use ID to get audio features per song
    Append every time
4. Append to a list of dictionary
5. Convert to Dataframe - save every 100 
6. Clean and Munge
7. Clustering - Create Labels for clusters 
    PCA

PICKLE!!!

In [1]:
import pandas as pd
import numpy as np

In [2]:
# import CSV of Top Karaoke Songs
tks = pd.read_excel('/Users/andrewjeong/Downloads/TopKaraokeSongs.xlsx')
tks.head()

,Title,Artist
0,Let It Go,Frozen
1,Happy,Pharrell Williams
2,Uptown Funk,Mark Ronson feat. Bruno Mars
3,Shake It off,Taylor Swift
4,All About That Bass,Meghan Trainor


In [3]:
# Import Spotipy (Spotify Wrapper) and fuction returning spotify ID using artist and track
import spotipy
sp = spotipy.Spotify()

def extract_id(artist, title):
    results = sp.search(q=artist+" "+title) 
    for i, t in enumerate(results['tracks']['items']):
        return t['id']

In [4]:
# create list of spotify ID's
ids = []
for x in tks.index:
    ids.append(extract_id(tks["Artist"][x], tks["Title"][x]))
ids=np.array(ids)

In [5]:
# Add column id in tks
tks['id'] = ids

In [6]:
tks.head()

,Title,Artist,id
0,Let It Go,Frozen,0qcr5FMsEO85NAQjrlDRKo
1,Happy,Pharrell Williams,6NPVjNh8Jhru9xOmyQigds
2,Uptown Funk,Mark Ronson feat. Bruno Mars,5MjukdbL11ysj7TnEM2GPn
3,Shake It off,Taylor Swift,1j2sRZpFUzG6wPJGfRvet3
4,All About That Bass,Meghan Trainor,0ifSeVGUr7py5GggttDhXw


In [10]:
# Get features for each song in tks

from __future__ import print_function    # (at top of module)
from spotipy.oauth2 import SpotifyClientCredentials
import json
import spotipy
import time
import sys

client_credentials_manager = SpotifyClientCredentials(client_id='9342202d7b4640c8aa49d83b62b570ba', client_secret = '9c08dfa4279348dda0d13d410a762728')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
sp.trace=False

features = sp.audio_features(tks['id'])
features

[{u'acousticness': 0.556,
  u'analysis_url': u'https://api.spotify.com/v1/audio-analysis/0qcr5FMsEO85NAQjrlDRKo',
  u'danceability': 0.539,
  u'duration_ms': 223840,
  u'energy': 0.488,
  u'id': u'0qcr5FMsEO85NAQjrlDRKo',
  u'instrumentalness': 0,
  u'key': 8,
  u'liveness': 0.122,
  u'loudness': -6.864,
  u'mode': 1,
  u'speechiness': 0.0318,
  u'tempo': 137.073,
  u'time_signature': 4,
  u'track_href': u'https://api.spotify.com/v1/tracks/0qcr5FMsEO85NAQjrlDRKo',
  u'type': u'audio_features',
  u'uri': u'spotify:track:0qcr5FMsEO85NAQjrlDRKo',
  u'valence': 0.373},
 {u'acousticness': 0.286,
  u'analysis_url': u'https://api.spotify.com/v1/audio-analysis/6NPVjNh8Jhru9xOmyQigds',
  u'danceability': 0.652,
  u'duration_ms': 233305,
  u'energy': 0.757,
  u'id': u'6NPVjNh8Jhru9xOmyQigds',
  u'instrumentalness': 0,
  u'key': 1,
  u'liveness': 0.0886,
  u'loudness': -6.819,
  u'mode': 1,
  u'speechiness': 0.153,
  u'tempo': 159.911,
  u'time_signature': 4,
  u'track_href': u'https://api.spotif

In [12]:
# convert features to dataframe
af = pd.DataFrame(features)

In [14]:
af.head()

,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,0.5560,https://api.spotify.com/v1/audio-analysis/0qcr...,0.539,223840,0.488,0qcr5FMsEO85NAQjrlDRKo,0.000000,8,0.1220,-6.864,1,0.0318,137.073,4,https://api.spotify.com/v1/tracks/0qcr5FMsEO85...,audio_features,spotify:track:0qcr5FMsEO85NAQjrlDRKo,0.373
1,0.2860,https://api.spotify.com/v1/audio-analysis/6NPV...,0.652,233305,0.757,6NPVjNh8Jhru9xOmyQigds,0.000000,1,0.0886,-6.819,1,0.1530,159.911,4,https://api.spotify.com/v1/tracks/6NPVjNh8Jhru...,audio_features,spotify:track:6NPVjNh8Jhru9xOmyQigds,0.962
2,0.0417,https://api.spotify.com/v1/audio-analysis/5Mju...,0.823,271360,0.498,5MjukdbL11ysj7TnEM2GPn,0.000000,0,0.1070,-7.257,1,0.1020,115.046,4,https://api.spotify.com/v1/tracks/5MjukdbL11ys...,audio_features,spotify:track:5MjukdbL11ysj7TnEM2GPn,0.638
3,0.0031,https://api.spotify.com/v1/audio-analysis/1j2s...,0.598,217602,0.909,1j2sRZpFUzG6wPJGfRvet3,0.004990,6,0.0734,-4.686,1,0.0559,160.046,4,https://api.spotify.com/v1/tracks/1j2sRZpFUzG6...,audio_features,spotify:track:1j2sRZpFUzG6wPJGfRvet3,0.966
4,0.0476,https://api.spotify.com/v1/audio-analysis/0ifS...,0.811,188754,0.879,0ifSeVGUr7py5GggttDhXw,0.000002,9,0.1090,-3.724,1,0.0514,134.064,4,https://api.spotify.com/v1/tracks/0ifSeVGUr7py...,audio_features,spotify:track:0ifSeVGUr7py5GggttDhXw,0.962


In [15]:
# Join dataframe tks (karaoke songs) and dataframe features on id

af = pd.merge(tks,af, on=['id','id'])
af.head()

,Title,Artist,id,acousticness,analysis_url,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,Let It Go,Frozen,0qcr5FMsEO85NAQjrlDRKo,0.5560,https://api.spotify.com/v1/audio-analysis/0qcr...,0.539,223840,0.488,0.000000,8,0.1220,-6.864,1,0.0318,137.073,4,https://api.spotify.com/v1/tracks/0qcr5FMsEO85...,audio_features,spotify:track:0qcr5FMsEO85NAQjrlDRKo,0.373
1,Happy,Pharrell Williams,6NPVjNh8Jhru9xOmyQigds,0.2860,https://api.spotify.com/v1/audio-analysis/6NPV...,0.652,233305,0.757,0.000000,1,0.0886,-6.819,1,0.1530,159.911,4,https://api.spotify.com/v1/tracks/6NPVjNh8Jhru...,audio_features,spotify:track:6NPVjNh8Jhru9xOmyQigds,0.962
2,Uptown Funk,Mark Ronson feat. Bruno Mars,5MjukdbL11ysj7TnEM2GPn,0.0417,https://api.spotify.com/v1/audio-analysis/5Mju...,0.823,271360,0.498,0.000000,0,0.1070,-7.257,1,0.1020,115.046,4,https://api.spotify.com/v1/tracks/5MjukdbL11ys...,audio_features,spotify:track:5MjukdbL11ysj7TnEM2GPn,0.638
3,Shake It off,Taylor Swift,1j2sRZpFUzG6wPJGfRvet3,0.0031,https://api.spotify.com/v1/audio-analysis/1j2s...,0.598,217602,0.909,0.004990,6,0.0734,-4.686,1,0.0559,160.046,4,https://api.spotify.com/v1/tracks/1j2sRZpFUzG6...,audio_features,spotify:track:1j2sRZpFUzG6wPJGfRvet3,0.966
4,All About That Bass,Meghan Trainor,0ifSeVGUr7py5GggttDhXw,0.0476,https://api.spotify.com/v1/audio-analysis/0ifS...,0.811,188754,0.879,0.000002,9,0.1090,-3.724,1,0.0514,134.064,4,https://api.spotify.com/v1/tracks/0ifSeVGUr7py...,audio_features,spotify:track:0ifSeVGUr7py5GggttDhXw,0.962


In [16]:
# pickle dataframe
import pickle

with open('karaoke_songs.pkl', 'w') as ks:
    pickle.dump(af, ks)

In [18]:
# open pickled dataframe
with open('karaoke_songs.pkl', 'r') as ks:
    af = pickle.load(ks)

In [23]:
# check unique values for column 'type'
af['type'].unique()

array([u'audio_features'], dtype=object)

In [24]:
# check unique values for column 'mode'
af['mode'].unique()

array([1, 0])

In [26]:
# drop columns ('analysis_url','track_href','type','uri')
af.drop(['analysis_url','track_href','type','uri'],axis=1,inplace=True)

In [27]:
# check types for columns
af.dtypes

Title                object
Artist               object
id                   object
acousticness        float64
danceability        float64
duration_ms           int64
energy              float64
instrumentalness    float64
key                   int64
liveness            float64
loudness            float64
mode                  int64
speechiness         float64
tempo               float64
time_signature        int64
valence             float64
dtype: object

In [20]:
# Describe 
af.describe()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
count,100.000000,100.000000,100.0000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.00000
mean,0.369019,0.581400,226551.4300,0.558500,0.071015,5.150000,0.164754,-8.710270,0.820000,0.063429,120.986490,3.930000,0.54614
std,0.319913,0.168228,58134.7549,0.234455,0.224561,3.505767,0.121372,4.224065,0.386123,0.061148,29.423366,0.408372,0.24382
min,0.000453,0.225000,68453.0000,0.077400,0.000000,0.000000,0.056400,-19.885000,0.000000,0.025100,66.980000,1.000000,0.10800
25%,0.047300,0.472000,194967.0000,0.374750,0.000000,2.000000,0.089100,-10.974750,1.000000,0.033375,99.961000,4.000000,0.34650
50%,0.290000,0.563500,219486.5000,0.588000,0.000006,5.000000,0.115000,-7.517000,1.000000,0.042350,119.530000,4.000000,0.55250
75%,0.635250,0.730750,257465.2500,0.774000,0.000308,8.000000,0.186250,-5.839500,1.000000,0.059650,134.585000,4.000000,0.76325
max,0.938000,0.978000,515867.0000,0.939000,0.966000,11.000000,0.680000,-2.451000,1.000000,0.453000,213.840000,5.000000,1.00000


In [29]:
# Correlation between 
af.corr()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
acousticness,1.000000,-0.395794,0.000916,-0.758609,-0.025282,0.006294,0.047898,-0.596870,0.110520,0.059108,-0.108968,-0.276113,-0.318598
danceability,-0.395794,1.000000,-0.121157,0.331420,0.171597,-0.045010,-0.133904,0.213618,-0.182219,0.173167,-0.128482,0.130829,0.576476
duration_ms,0.000916,-0.121157,1.000000,0.009259,0.051522,0.064614,-0.145737,-0.052310,0.008949,-0.138425,0.045021,0.052375,-0.225801
energy,-0.758609,0.331420,0.009259,1.000000,-0.139103,-0.016928,0.049192,0.775581,-0.113810,0.035345,0.083711,0.221601,0.383232
instrumentalness,-0.025282,0.171597,0.051522,-0.139103,1.000000,0.138505,-0.136141,-0.301811,-0.200492,-0.015568,-0.076761,0.054669,0.141483
key,0.006294,-0.045010,0.064614,-0.016928,0.138505,1.000000,0.061130,-0.081019,-0.039549,-0.095140,-0.105081,-0.119590,-0.019322
liveness,0.047898,-0.133904,-0.145737,0.049192,-0.136141,0.061130,1.000000,-0.004183,0.008421,0.106294,0.015283,-0.326421,0.070583
loudness,-0.596870,0.213618,-0.052310,0.775581,-0.301811,-0.081019,-0.004183,1.000000,-0.016882,-0.038871,-0.082027,0.214758,0.093988
mode,0.110520,-0.182219,0.008949,-0.113810,-0.200492,-0.039549,0.008421,-0.016882,1.000000,-0.156014,0.109442,0.175523,-0.094898
speechiness,0.059108,0.173167,-0.138425,0.035345,-0.015568,-0.095140,0.106294,-0.038871,-0.156014,1.000000,0.185898,0.078516,0.253072


In [31]:
# Check for nulls
af[pd.isnull(af).any(axis=1)]

,Title,Artist,id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence


In [30]:
import seaborn as sns

sns.heatmap(af)

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''